In [1]:
import os
os.chdir("../src")
%pwd

'/home/jovyan/work/sentiment_analysis_in_russian_news_feeds/src'

In [44]:
from pathlib import Path
from russian_news_sentiment_analysis.utils.common import init_spark_session, load_txt, read_yaml
from russian_news_sentiment_analysis.config.configuration import ConfigurationManager

manager = ConfigurationManager()

spark = init_spark_session()

[2024-03-30 07:35:27,275: 67: logger: INFO: common:  yaml file: russian_news_sentiment_analysis/config/config.yaml loaded successfully]


In [45]:
import pandas as pd
from tqdm.auto import tqdm

models = [k['model_name'] for k in read_yaml(Path(manager.config.path.models_params)).values()]
datasets = ['.'.join(k['filename'].split('.')[:-1]) for k in read_yaml(Path(manager.config.path.datasets_params)).values()]
metrics = ['accuracy', 'weightedPrecision', 'weightedRecall', 'fMeasureByLabel_1', 'weightedFMeasure', 'fMeasureByLabel_0']
dfs = dict([(m, pd.DataFrame(columns=datasets, index=models)) for m in metrics])

[2024-03-30 07:35:27,440: 67: logger: INFO: common:  yaml file: /home/jovyan/work/sentiment_analysis_in_russian_news_feeds/src/russian_news_sentiment_analysis/params/models.yaml loaded successfully]
[2024-03-30 07:35:27,452: 67: logger: INFO: common:  yaml file: /home/jovyan/work/sentiment_analysis_in_russian_news_feeds/src/russian_news_sentiment_analysis/params/datasets.yaml loaded successfully]


In [46]:
for dataset in tqdm(datasets, total=len(datasets)):
    for model in models:
        path = f"{dataset}_{model.replace('/', '--')}.parquet"
        df = spark.read.json(os.path.join(manager.config.path.metrics_data, path)).toPandas()
        for metric in metrics:
            dfs[metric].loc[model, dataset] = round(df[df['metric'] == metric]['value'].item(),4)

  0%|          | 0/8 [00:00<?, ?it/s]

In [47]:
for (k, v) in dfs.items():
    v.to_csv(f'/home/jovyan/work/sentiment_analysis_in_russian_news_feeds/metrics/{k}.csv')

In [48]:
spark.stop()